In [1]:
import sys
sys.path.append('..')

import attack

In [2]:
import geopandas as gp
import pandas as pd
from tqdm import tqdm
import numpy as np
import libpysal
import itertools
from joblib import Parallel, delayed
import random


In [3]:
# load data frames (~ 40 secs)
import load_data_module

In [4]:
# Load preprocessed data
raw_full_trip_gdf, raw_trip_sp_gdf, raw_trip_ep_gdf, tesselation_gdf = load_data_module.raw_full_trip_gdf, load_data_module.raw_trip_sp_gdf, load_data_module.raw_trip_ep_gdf, load_data_module.tesselation_gdf
assert len(raw_full_trip_gdf) == len(raw_trip_sp_gdf) == len(raw_trip_ep_gdf)

In [5]:
# # Select n random person ids from the dataset
# n_users = 4
# raw_full_trip_gdf, raw_trip_sp_gdf, raw_trip_ep_gdf = attack.select_n_random_users_from_dataframes(n_users, raw_full_trip_gdf, raw_trip_sp_gdf, raw_trip_ep_gdf)

## Evaluation Functions

# Merge Start Points (SP) and End Points (EP) with Tessellation

In [6]:
gdf_sp, gdf_ep = attack.match_boundary_points_with_tessellation(raw_trip_sp_gdf, raw_trip_ep_gdf, tesselation_gdf)

# Extract Full Trips that Start and End within Tessellation Area

In [7]:
full_trip_gdf, trip_sp_gdf, trip_ep_gdf, gdf_sp, gdf_ep = attack.extract_trips_that_start_end_in_tessellation(raw_full_trip_gdf, raw_trip_sp_gdf, raw_trip_ep_gdf, gdf_sp, gdf_ep)


Number of trips that start and end wihin tessellation area: 996
Number of trips outside and therefore dropped: 28


## Build mapping of trip chains

In [8]:
mapping_cont_trips = attack.build_trip_chain_mapping(gdf_sp, gdf_ep)

100%|██████████| 996/996 [00:17<00:00, 57.09it/s]


In [9]:
attack.evaluate_trip_chaining(mapping_cont_trips, full_trip_gdf)

Number of edges (matched) between trips: 340
Number of wrong matches: 12


## Merge trips according to matching

In [10]:
full_trips_concat_gdf, trip_concat_dict = attack.merge_trips_from_matching(gdf_sp, mapping_cont_trips, full_trip_gdf)

Building trip chains...


100%|██████████| 996/996 [00:00<00:00, 10060.68it/s]


Done.
Merging trips...


100%|██████████| 996/996 [00:14<00:00, 69.70it/s] 


Done.
Number of trips that were matched at least once: 985/996
Concatenating MERGED and UNMERGED trips...
Done.


In [11]:
gdf_sp_concat, trip_sp_gdf_concat, gdf_ep_concat, trip_ep_gdf_concat = attack.extract_concatenated_trips(full_trips_concat_gdf, gdf_sp, trip_sp_gdf, gdf_ep, trip_ep_gdf)

## Build Clustering after Concatenation Step

In [12]:
clustering_concat = attack.build_clustering_after_concatenation(full_trips_concat_gdf, trip_concat_dict, full_trip_gdf)

Number of unique clusters: 667


# Build Home Locations (HL)

## From Start Points (SPs)

We use the SP-tessellation matching that still contains all SP (and potential HL), and not just the SP and EP of the concatenated trips. We do this, because we do not want loose potential HL contributed of substrip concatenated in a chain.

In [13]:
gdf_hl_combined_sp = attack.build_hl_from_start_points(gdf_sp)

c:\Users\Bened\Documents\Git\Master-Thesis\env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 60 disconnected components.
 There are 36 islands with ids: 1, 2, 20, 21, 28, 29, 30, 31, 35, 36, 37, 38, 39, 43, 44, 54, 55, 62, 63, 64, 66, 69, 77, 78, 79, 80, 81, 84, 93, 94, 112, 113, 130, 133, 134, 135.
  warnings.warn(message)


## From End Points (EPs)

In [14]:
gdf_hl_combined_ep = attack.build_hl_from_end_points(gdf_ep)

c:\Users\Bened\Documents\Git\Master-Thesis\env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 104 disconnected components.
 There are 40 islands with ids: 4, 20, 27, 38, 39, 68, 80, 85, 86, 93, 94, 95, 96, 98, 99, 107, 112, 113, 114, 123, 126, 133, 134, 137, 152, 159, 173, 175, 178, 179, 196, 197, 218, 228, 251, 259, 261, 271, 286, 290.
  warnings.warn(message)


## Merge (concatenate)

In [15]:
gp_combined, HL_table = attack.concatenate_hl(gdf_hl_combined_sp, gdf_hl_combined_ep)

c:\Users\Bened\Documents\Git\Master-Thesis\env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 126 disconnected components.
 There are 53 islands with ids: 4, 29, 52, 78, 79, 126, 146, 147, 154, 155, 164, 176, 177, 179, 189, 190, 191, 194, 209, 216, 225, 226, 229, 240, 254, 264, 277, 284, 285, 295, 302, 303, 315, 316, 342, 343, 348, 349, 354, 356, 363, 374, 375, 377, 390, 392, 394, 397, 405, 406, 424, 425, 444.
  warnings.warn(message)


Count unique PERSON_IDs per HL:  PERSON_ID
1            89
2            28
3             6
4             2
5             1
dtype: int64
Number of users for which at least on Home Location has been identified:  54
Number of unique HL tiles: 126


# Match trips with Home Location tiles

## Match concatenated trips

In [16]:
HL_table_se_concat, unmatched_trips, double_assigned_trips, nr_unmatched = attack.match_trips_to_HL(gp_combined, HL_table, trip_sp_gdf_concat, trip_ep_gdf_concat, full_trips_concat_gdf)

Number of unmatched trajectories (concatenated) that do neither start nor end in a HL tile: 88/667
Number of trajectories (concatenated) that start AND end in a HL tile: 337/667
Number of trips that match different HL tiles with their SP and EP: 164


## Assign double matched trips to one unique HL

Iterate over all double matched trips and compare them to all other trips that are uniquely assigned in their respective potential HLs that they have been matched with. Then take the HL with the single maximum lcss score between the trip under question and any trip of the assigned HL tiles.

In [17]:
HL_table_trips_concat = attack.assign_double_matched_trips_to_unique_hl(HL_table_se_concat, full_trips_concat_gdf, unmatched_trips, double_assigned_trips, nr_unmatched)


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   40.3s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-2)]: Done 167 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-2)]: Done 186 tasks      | elapsed:   

## Get trajectories that happened during the same time

In [18]:
full_trips_concat_gdf_overlap_dict = attack.getTripOverlaps(full_trips_concat_gdf)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-2)]: Done 167 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-2)]: Done 186 tasks      | elapsed:   

## Create clustering after HL assignment step

In [28]:
def getOverlappingTrips(traj_id_list, full_trips_concat_gdf_overlap_dict):
    """This function finds the overlapping trips for a list of trajectory IDs.

    Args:
        traj_id_list (_type_): List of trajectory IDs.
    """
    overlapping_trips = [item for sublist in [full_trips_concat_gdf_overlap_dict[t] for t in traj_id_list] for item in sublist] # we first get a list of lists and then flatten it
    return overlapping_trips

def findLargestNonSimultaneousSubset(traj_id_list, full_trips_concat_gdf_overlap_dict, RANDOMIZED_SEARCH_THRESHOLD=30, RANDOMIZED_SEARCH_ITERATIONS=1000):
    """This function finds the largest subset of trajectories that are not simultaneous. It uses a determinitic algorithm if the length of the trajectory ID list is smaller than a threshold and a randomized algorithm if the length is larger than the given threshold.

    Args:
        traj_id_list (list): List of trajectory IDs.

    Returns:
        list: List of trajectory IDs that are not simultaneous.
    """
    len_traj_id_list = len(traj_id_list)

    # Deterministic (optimal) algorithm
    if len_traj_id_list <= RANDOMIZED_SEARCH_THRESHOLD:
        # We create a list of all possible subsets of the trajectory ID list with decreasing length
        # We do this iteratively to not overload the memory
        for i in range(len_traj_id_list):
            # Create a list of all possible subsets of the trajectory ID list with length len_traj_id_list - i
            subsets = list(itertools.combinations(traj_id_list, len_traj_id_list - i))
        
            # Sort the list by length of the subsets
            subsets.sort(key=len, reverse=True)

            # Loop through the list of subsets
            for subset in subsets:
                # get all trips that do overlap in time with any of the trips in subset
                overlapping_trips = getOverlappingTrips(subset, full_trips_concat_gdf_overlap_dict)

                # Check if the subset is not simultaneous
                if all([t not in overlapping_trips for t in subset]):
                    # If so, return the subset as list
                    return list(subset)

    # Randomized algorithm (numeric approximation)
    else:
        def randomized_subset_search(traj_id_list):
            subsets = []
            candidates = traj_id_list.copy()
            id = candidates.pop(random.randrange(len(candidates)))
            subset = [id]
            while candidates:
                next = candidates.pop(random.randrange(len(candidates)))

                if all([t not in getOverlappingTrips(subset, full_trips_concat_gdf_overlap_dict) for t in subset]):
                    subset.append(next)
            
            subsets.append(subset)
            return max(subsets, key=len)
        
        # We run the randomized subset search 100 times and return the longest subset
        print(f'Running randomized subset for search for {RANDOMIZED_SEARCH_ITERATIONS} iterations with {len(traj_id_list)} trajectories...')
        result = Parallel(n_jobs=-2, verbose=10)(delayed(randomized_subset_search)(traj_id_list) for _ in range(RANDOMIZED_SEARCH_ITERATIONS))
        print('Done. Length of longest subset: ', len(max(result, key=len)))
        return max(result, key=len)


def build_clustering_after_HL_assignment(HL_table_trips_concat, full_trip_gdf, trip_concat_dict, full_trips_concat_gdf_overlap_dict):
    # This creates the array with clustering IDs after the HL assignment step
    clustering_after_HL = {}
    HL_table_dict = (HL_table_trips_concat.groupby('HL_ID')
        .apply(lambda x: list(dict(x.TRIP_ID).values()))
        .to_dict())

    for index, HL in tqdm(enumerate(HL_table_dict), total=len(HL_table_dict)):
        # Skip HL_ID -1, we will assign a clustering ID to these trips later
        if HL == -1:
            continue

        print(len(HL_table_dict[HL]))

        # find the largest subset of trips that are not simultaneous
        non_simultaneous_subset = findLargestNonSimultaneousSubset(HL_table_dict[HL], full_trips_concat_gdf_overlap_dict)

        # assign hl_id -1 to all trips that are not part of the largest subset
        for trip in HL_table_dict[HL]:
            if trip not in non_simultaneous_subset:
                HL_table_dict[-1].append(trip)

        # Loop through all trips that are assigned to this HL_ID and assign the same clustering ID to all of them
        for trip in non_simultaneous_subset:
            clustering_after_HL[attack.getIndexInList(trip, full_trip_gdf)] = index

            # Check if this trip is a concatenated trip and assign the same clustering ID to all trips that are part of the concatenated trip
            if trip in trip_concat_dict:
                for t in trip_concat_dict[trip]:
                    clustering_after_HL[attack.getIndexInList(t, full_trip_gdf)] = index

    # Assign clustering IDs to all hl_id -1 trips (these are the trips that were not successfully assigned to any HL_ID)
    for index, unm_trip in enumerate(HL_table_dict[-1]):
        clustering_after_HL[attack.getIndexInList(unm_trip, full_trip_gdf)] = index + len(HL_table_dict) - 1 # -1 because we don't want to count the -1 HL_ID in the length of HL_table_dict

        # Check if this trip is a concatenated trip and assign the same clustering ID to all trips that are part of the concatenated trip
        if unm_trip in trip_concat_dict:
            for t in trip_concat_dict[unm_trip]:
                clustering_after_HL[attack.getIndexInList(t, full_trip_gdf)] = index + len(HL_table_dict) - 1

    return clustering_after_HL, HL_table_dict

In [35]:
clustering_after_HL, HL_table_dict = build_clustering_after_HL_assignment(HL_table_trips_concat, 
    full_trip_gdf, 
    trip_concat_dict, 
    full_trips_concat_gdf_overlap_dict)

 16%|█▋        | 14/86 [00:00<00:00, 131.65it/s]

7
1
1
1
1
18
10
4
7
15
4
4
2
1
1
1
2
24
1
3
2
23
65
Running randomized subset for search for 1000 iterations with 65 trajectories...


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.1652s.) Setting batch_size=2.
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0177s.) Setting batch_size=4.
[Parallel(n_jobs=-2)]: Done  52 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0161s.) Setting batch_size=8.
[Parallel(n_jobs=-2)]: Done  78 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Done 134 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0593s.) Setting batch_size=16.
[Parallel(n_jobs=-2)]: Done 238 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Batch comp

Done. Length of longest subset:  27
1
3
1
3
17
6
6
6
1
1
5
8
1
9
11
2
2
4
3
1
12
4
2
4
36
Running randomized subset for search for 1000 iterations with 36 trajectories...


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0050s.) Setting batch_size=2.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0173s.) Setting batch_size=4.
[Parallel(n_jobs=-2)]: Done  22 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0180s.) Setting batch_size=8.
[Parallel(n_jobs=-2)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0560s.) Setting batch_size=16.
[Parallel(n_jobs=-2)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0688s.) Setting batch_size=32.
[Parallel(n_jobs=-2)]: Done 242 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done 466 tasks      | elapsed:    0.2s
[Parallel(n

Done. Length of longest subset:  26
4
1
3
2
2
3
6
2
1
5
3
1
3
5
23
12
1
22
19
1
3
1
9
13
4
23


100%|██████████| 86/86 [00:03<00:00, 24.70it/s]


2
6
9
10
12
2
1
1
1
8
8


In [30]:
print("Clustering results after concatenation step:")
print(f"Number of unique clusters: {len(set(clustering_concat))}")
attack.evaluate(clustering_concat, full_trip_gdf)

clustering_HL = list(dict(sorted(clustering_after_HL.items())).values())
print("\nClustering results after HL matching step:")
print(f"Number of unique clusters: {len(set(clustering_HL))}")
attack.evaluate(clustering_HL, full_trip_gdf)

Clustering results after concatenation step:
Number of unique clusters: 667
Homogeneity: 0.995
Completeness: 0.578
V-measure: 0.731
Rand index: 0.971
ARI: 0.061
MI: 3.684
NMI: 0.731
AMI: 0.267

Clustering results after HL matching step:
Number of unique clusters: 227
Homogeneity: 0.908
Completeness: 0.739
V-measure: 0.814
Rand index: 0.978
ARI: 0.524
MI: 3.362
NMI: 0.814
AMI: 0.691


## Assign Trips Without Match

In [31]:
clustering_after_double_assign_HL = attack.assign_trips_without_match(
    clustering_after_HL, HL_table_dict, 
    full_trips_concat_gdf, 
    full_trips_concat_gdf_overlap_dict, 
    full_trip_gdf, 
    trip_concat_dict,
    SIM_THRESH_FOR_NO_MATCH=0.25)


Comparing trips that were not assigned to any HL_ID with trips that were assigned to a HL_ID...


  1%|          | 1/143 [00:05<12:30,  5.29s/it]

existing match and assign cluster id 111.0 to trip 985302


  1%|▏         | 2/143 [00:27<36:03, 15.34s/it]

existing match and assign cluster id 18.0 to trip 988341


...

trip overlaps in time with other trips in the same HL_ID


 66%|██████▌   | 94/143 [24:07<12:27, 15.26s/it]

trip overlaps in time with other trips in the same HL_ID


 66%|██████▋   | 95/143 [24:23<12:19, 15.41s/it]

existing match and assign cluster id 87.0 to trip 1005502
Done.
Assigning clustering IDs to all trips that are part of a new cluster...
Done.


In [33]:
print("Clustering results after concatenation step:")
print(f"Number of unique clusters: {len(set(clustering_concat))}")
attack.evaluate(clustering_concat, full_trip_gdf)

print("\nClustering results after double assign HL step:")
print(f"Number of unique clusters: {len(set(list(dict(sorted(clustering_after_double_assign_HL.items())).values())))}")
attack.evaluate(list(dict(sorted(clustering_after_double_assign_HL.items())).values()), full_trip_gdf)

Clustering results after concatenation step:
Number of unique clusters: 667
Homogeneity: 0.995
Completeness: 0.578
V-measure: 0.731
Rand index: 0.971
ARI: 0.061
MI: 3.684
NMI: 0.731
AMI: 0.267

Clustering results after double assign HL step:
Number of unique clusters: 188
Homogeneity: 0.895
Completeness: 0.755
V-measure: 0.819
Rand index: 0.978
ARI: 0.544
MI: 3.317
NMI: 0.819
AMI: 0.711


In [36]:
clustering_after_double_assign_HL = attack.assign_trips_without_match(
    clustering_after_HL, HL_table_dict, 
    full_trips_concat_gdf, 
    full_trips_concat_gdf_overlap_dict, 
    full_trip_gdf, 
    trip_concat_dict,
    SIM_THRESH_FOR_NO_MATCH=0.5)


Comparing trips that were not assigned to any HL_ID with trips that were assigned to a HL_ID...


  1%|▏         | 2/148 [00:18<24:53, 10.23s/it]

trip overlaps in time with other trips in the same HL_ID


  2%|▏         | 3/148 [00:39<36:31, 15.12s/it]

existing match and assign cluster id 18.0 to trip 988341


...

trip overlaps in time with other trips in the same HL_ID


 74%|███████▎  | 109/148 [35:04<11:00, 16.94s/it]

trip overlaps in time with other trips in the same HL_ID


 75%|███████▌  | 111/148 [35:42<11:54, 19.30s/it]

trip overlaps in time with other trips in the same HL_ID
Done.
Assigning clustering IDs to all trips that are part of a new cluster...
Done.


In [37]:
print("Clustering results after concatenation step:")
print(f"Number of unique clusters: {len(set(clustering_concat))}")
attack.evaluate(clustering_concat, full_trip_gdf)

print("\nClustering results after double assign HL step:")
print(f"Number of unique clusters: {len(set(list(dict(sorted(clustering_after_double_assign_HL.items())).values())))}")
attack.evaluate(list(dict(sorted(clustering_after_double_assign_HL.items())).values()), full_trip_gdf)

Clustering results after concatenation step:
Number of unique clusters: 667
Homogeneity: 0.995
Completeness: 0.578
V-measure: 0.731
Rand index: 0.971
ARI: 0.061
MI: 3.684
NMI: 0.731
AMI: 0.267

Clustering results after double assign HL step:
Number of unique clusters: 205
Homogeneity: 0.903
Completeness: 0.747
V-measure: 0.817
Rand index: 0.978
ARI: 0.536
MI: 3.343
NMI: 0.817
AMI: 0.702
